In [167]:
import pandas as pd

In [168]:
from pyspark.sql.functions import *
from pyspark.sql import Window

In [169]:
from pyspark.sql import SparkSession

In [170]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("test") \
    .master("local[*]") \
    .getOrCreate()

In [171]:
file_path = "c:\\Users\\ninve\\Downloads\\CROSS_BORDER_COMMERCIAL_SCHEDULE_201501010000-201601010000_BG.xlsx"

In [172]:
df = pd.read_excel(file_path, header = [3,4,5], na_values=['N/E', 'Nan']).dropna(how='all')

In [173]:
df.columns = [col[1].replace(' ', "_") if col[0].startswith("Unnamed") else col[0].replace(' > ', '_') + '_' + col[1].replace('-', '_') + '_' + col[2].replace(r' \[|\] ', '')  for col in df.columns]

In [174]:
spark_df = spark.createDataFrame(df)

In [175]:
spark_df = spark_df.withColumn("raw_date", when(col("Time_Interval").contains('.'), col("Time_Interval")))

In [176]:
w = Window.orderBy(monotonically_increasing_id()).rowsBetween(Window.unboundedPreceding, 0)
spark_df = spark_df.withColumn("r_date", last("raw_date", ignorenulls=True).over(w))
spark_df = spark_df.withColumn("date", when(col("r_date").isNull(), '01.01.2015').otherwise(col("r_date")))
spark_df = spark_df.withColumn("date", to_date(col("date"), 'dd.MM.yyyy')) \
    .filter(col("Time_Interval").contains('-')).drop("r_date", "raw_date")

In [177]:
id_cols = ["date", "Time_Interval"]
value_cols = [c  for c in spark_df.columns if c not in id_cols]

In [178]:
pd_df = spark_df.toPandas()

In [179]:
df_long = pd_df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name='Direction',
    value_name='MW'
)

In [180]:
sp_df = spark.createDataFrame(df_long)

In [181]:
sp_df.groupBy("Date", "Direction").agg(sum('MW').alias("MW_per_month")).orderBy("date").dropna().show()

+----------+--------------------+------------+
|      Date|           Direction|MW_per_month|
+----------+--------------------+------------+
|2015-01-01| RS_BG_Intraday_[MW]|       103.0|
|2015-01-01|MK_BG_Day_ahead_[MW]|         0.0|
|2015-01-01|GR_BG_Day_ahead_[MW]|         0.0|
|2015-01-01|BG_GR_Day_ahead_[MW]|     14400.0|
|2015-01-01|BG_MK_Day_ahead_[MW]|      4579.0|
|2015-01-01|RS_BG_Day_ahead_[MW]|       103.0|
|2015-01-01|BG_RS_Day_ahead_[MW]|      3353.0|
|2015-01-01| BG_RS_Intraday_[MW]|      3353.0|
|2015-01-02|GR_BG_Day_ahead_[MW]|         0.0|
|2015-01-02|BG_GR_Day_ahead_[MW]|     14400.0|
|2015-01-02|MK_BG_Day_ahead_[MW]|         0.0|
|2015-01-02| RS_BG_Intraday_[MW]|       290.0|
|2015-01-02|BG_MK_Day_ahead_[MW]|      4819.0|
|2015-01-02|RS_BG_Day_ahead_[MW]|       290.0|
|2015-01-02|BG_RS_Day_ahead_[MW]|      3644.0|
|2015-01-02| BG_RS_Intraday_[MW]|      3644.0|
|2015-01-03|GR_BG_Day_ahead_[MW]|         0.0|
|2015-01-03|BG_MK_Day_ahead_[MW]|      5013.0|
|2015-01-03|B